# **Notebook 13.4: Graph attention networks**

This notebook builds a graph attention mechanism from scratch, as discussed in section 13.8.6 of the book and illustrated in figure 13.12c

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

The self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [ ]:
# Set seed so we get the same random numbers
np.random.seed(1)
# Number of nodes in the graph
N = 8
# Number of dimensions of each input
D = 4

# Define a graph
A = np.array([[0,1,0,1,0,0,0,0],
              [1,0,1,1,1,0,0,0],
              [0,1,0,0,1,0,0,0],
              [1,1,0,0,1,0,0,0],
              [0,1,1,1,0,1,0,1],
              [0,0,0,0,1,0,1,1],
              [0,0,0,0,0,1,0,0],
              [0,0,0,0,1,1,0,0]]);
print(A)

# Let's also define some random data
X = np.random.normal(size=(D,N))

[[0 1 0 1 0 0 0 0]
 [1 0 1 1 1 0 0 0]
 [0 1 0 0 1 0 0 0]
 [1 1 0 0 1 0 0 0]
 [0 1 1 1 0 1 0 1]
 [0 0 0 0 1 0 1 1]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 1 1 0 0]]


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [ ]:
# Choose random values for the parameters
omega = np.random.normal(size=(D,D))
beta = np.random.normal(size=(D,1))
phi = np.random.normal(size=(1,2*D))

We'll need a softmax operation that operates on the columns of the matrix and a ReLU function as well

In [ ]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax


# Define the Rectified Linear Unit (ReLU) function
def ReLU(preactivation):
  activation = preactivation.clip(0.0)
  return activation


In [ ]:
 # Now let's compute self attention in matrix form
def graph_attention(X,omega, beta, phi, A):

  # TODO -- Write this function (see figure 13.12c)
  # 1. Compute X_prime
  # 2. Compute S
  # 3. To apply the mask, set S to a very large negative number (e.g. -1e20) everywhere where A+I is zero
  # 4. Run the softmax function to compute the attention values
  # 5. Postmultiply X' by the attention values
  # 6. Apply the ReLU function
  # Replace this line:
  output = np.ones_like(X) ;

  # BEGIN_ANSWER
  X_prime = beta * np.ones((1,N)) + omega @ X
  S = np.zeros((N,N))
  for c1 in range(N):
    for c2 in range(N):
      S[c1,c2] = phi @ (np.vstack([X[:,[c1]],X[:,[c2]]]))

  # print(S)
  mask = (A+np.identity(N) !=1) * np.ones((N,N))
  S = S - mask * S
  S = S - mask * 1e20
  attention = softmax_cols(S)
  output = ReLU(X_prime @ attention)

  # END_ANSWER
  return output;

In [ ]:
# Test out the graph attention mechanism
np.set_printoptions(precision=3)
output = graph_attention(X, omega, beta, phi, A);
print("Correct answer is:")
print("[[1.796 1.346 0.569 1.703 1.298 1.224 1.24  1.234]")
print(" [0.768 0.672 0.    0.529 3.841 4.749 5.376 4.761]")
print(" [0.305 0.129 0.    0.341 0.785 1.014 1.113 1.024]")
print(" [0.    0.    0.    0.    0.35  0.864 1.098 0.871]]]")


print("Your answer is:")
print(output)

Correct answer is:
[[1.796 1.346 0.569 1.703 1.298 1.224 1.24  1.234]
 [0.768 0.672 0.    0.529 3.841 4.749 5.376 4.761]
 [0.305 0.129 0.    0.341 0.785 1.014 1.113 1.024]
 [0.    0.    0.    0.    0.35  0.864 1.098 0.871]]]
Your answer is:
[[1.796 1.346 0.569 1.703 1.298 1.224 1.24  1.234]
 [0.768 0.672 0.    0.529 3.841 4.749 5.376 4.761]
 [0.305 0.129 0.    0.341 0.785 1.014 1.113 1.024]
 [0.    0.    0.    0.    0.35  0.864 1.098 0.871]]


TODO -- Try to construct a dot-product self-attention mechanism as in practical 12.1 that respects the geometry of the graph and has zero attention between non-neighboring nodes by combining figures 13.12a and 13.12b.

ANSWER: The self-attention mechanisms works as normal but we set all values of the pre-softmax attention to 1e-20 where A+I is zero.
